In [ ]:
!pip install git+https://github.com/Liadrinz/transformers-unilm
!pip install git+https://github.com/huggingface/transformers
!pip uninstall -y llmdet
!pip install --no-cache-dir git+https://github.com/dr-phil/LLMDetFix.git
!pip install scikit-learn transformers nltk lightgbm numpy dask[dataframe]
!pip install datasets

In [ ]:
# !pip install --no-cache-dir git+https://github.com/dr-phil/LLMDetFix.git
# !pip install git+https://github.com/dr-phil/LLMDetFix.git

In [ ]:
HF_KEY = "HUGGING_FACE_KEY_HERE"

# from google.colab import userdata
# userdata.set('HF_TOKEN', HF_KEY)

from huggingface_hub import login
login(HF_KEY)

In [ ]:
import llmdet

llmdet.load_probability()

text = "The actress was honoured for her role in 'The Reader' at the annual ceremony, which was held at the Royal Albert Hall. The film, which is based on the novel by the same name by Philip Roth, tells the story of a New York Times reporter who returns to his hometown to cover the death of his brother-in-law. Winslet plays his wife, with whom he has been divided since the death of their son.\nIn the film, Winslet plays the mother of the grieving brother-in-law.\nThe actress also won a Golden Globe for her role in the film at the ceremony in November.\nWinslet was also nominated for an Oscar for her role in 'The Reader'.\nThe 63-year-old Winslet was seen accepting her awards at the ceremony, where she was joined by her husband, John Krasinski, who has been nominated for best supporting actor in the film.\nWinslet and Krasinski met while"

# Detect, `text` is a string or string list
result = llmdet.detect(text)
print(result)

In [ ]:
ai_gen = []
ai_gen.append("Zimbabwe faces persistent economic hardships, including rampant inflation, an unstable currency system, and fundamental institutional problems, though government initiatives have occasionally attempted to address these issues.")
ai_gen.append("Zimbabwe's economic climate remains volatile, marked by persistent hyperinflation, currency instability, and high unemployment, despite government efforts to implement reforms and attract foreign investment.")

human_gen = []
human_gen.append("The economic climate of Zimbabwe is challenging to describe, given that a large majority of it is deemed an 'informal economy'.")
human_gen.append("Zimbabwe's formal economy mostly focuses on mining, with many state-subsidized enterprises supporting the sector. Other industries include textiles and machining.")

# Detect, `text` is a string or string list
res = []
# print(llmdet.detect(ai_gen[0])[0]['Human_write'])
res.append(llmdet.detect(ai_gen[0]))
res.append(llmdet.detect(ai_gen[1]))
res.append(llmdet.detect(human_gen[0]))
res.append(llmdet.detect(human_gen[1]))

print(res)

In [ ]:
for sublist in res:
    # Extract the dictionary from the sublist
    d = sublist[0]
    print(f"Human: {d['Human_write']}, AI: {1 - d['Human_write']}")
    max_key = max(d, key=d.get)
    print(f"Max key: {max_key}, Value: {d[max_key]}")

In [ ]:
import os
from datasets import load_dataset, load_from_disk

SAVE_PATH = '/content/data/wiki_dataset'

def get_wikipedia_dataset():
    # Check if dataset already exists on disk
    if os.path.exists(SAVE_PATH):
        print("Loading dataset from disk...")
        dataset = load_from_disk(SAVE_PATH)
    else:
        print("Dataset not found on disk. Downloading from Hugging Face...")
        # Create directory if it doesn't exist
        os.makedirs('/content/data', exist_ok=True)

        # Download and load the dataset
        dataset = load_dataset("wikipedia", "20220301.en")

        # Save to disk
        print("Saving dataset to disk...")
        dataset.save_to_disk(SAVE_PATH)

    print("Dataset is ready!")
    return dataset

In [ ]:
legacy_dataset = get_wikipedia_dataset()

In [ ]:
import json
from datasets import Dataset, DatasetDict

with open("/content/scraped_wiki_articles_8_2023.json", "r", encoding="utf-8") as file:
    data = json.load(file)

for entry in data:
    entry["text"] = entry.pop("content")

dataset = Dataset.from_list(data)

dataset_2023 = DatasetDict({"train": dataset})

print(dataset_2023)

In [8]:
import json
import re

full_scores = []
all_scores = []
non_ai = 0
ai = 0
def ai_score(lines, truncate_to=5000):
    print(f"number of lines: {len(lines)}")
    global non_ai, ai
    scores = []
    for ind, line in enumerate(lines):

        print(len(line))
        full_score = llmdet.detect(' '.join(line.strip().split()[:truncate_to]))[0]
        full_scores.append(full_score)
        score = full_score['Human_write']
        print(f" combined: {full_score} \n {score}, {ind}")
        scores.append((score, ind))

        if max(full_score, key=full_score.get) != 'Human_write' and score < 4e-3:
            print("~~~ ai article ~~~")
            ai += 1
        else:
            non_ai += 1

    return scores

def clean_text(text):
    # Replace all newlines with a unique string
    text = text.replace('\n', '!@#').replace('\t', ' ')
    # Split the text by the unique string
    split_text = text.split('!@#')
    # Remove elements with 5 or fewer words
    split_text = [segment for segment in split_text if len(segment.split()) > 5]
    # Rejoin the text
    text = ' '.join(split_text)
    # Remove "References" and "External links" sections
    text = re.sub(r'== References ==.*|== External links ==.*', '', text, flags=re.DOTALL)

    return text.strip()

def extract_title_text_and_url_legacy():
    titles, texts, urls = [], [], []

    for i in range(1000):
    #   title = legacy_dataset["train"][i]["title"]
    #   content = legacy_dataset["train"][i]["text"]
    #   url = legacy_dataset["train"][i]["url"]
    #   cleaned_text = clean_text(content)

    #   for new
      title = dataset_2023["train"][i]["title"]
      content = dataset_2023["train"][i]["text"]
      url = dataset_2023["train"][i]["url"]
      cleaned_text = clean_text(content)

      if title and cleaned_text:
          titles.append(title)
          texts.append(cleaned_text)
          urls.append(url)

    return titles, texts, urls

def process_files(file_path, output_file):
    titles, texts, urls = extract_title_text_and_url_legacy()
    texts = [' '.join(text.strip().split()[:400]) for text in texts]
    final_texts, final_titles, final_urls = [],[],[]
    for ind, text in enumerate(texts):
        if len(text.split()) >= 100:
            final_texts.append(texts[ind])
            final_titles.append(titles[ind])
            final_urls.append(urls[ind])

    print(f"number of texts: {len(texts)}")

    bino_scores = ai_score(texts)
    all_scores.append(bino_scores)

    with open(output_file, 'w', encoding='utf-8') as outfile:
        for title, score, url, text in zip(final_titles, bino_scores, final_urls, final_texts):
            outfile.write(f"{title}, {score[0]}\n")


In [ ]:
output_file = "/content/2023-wikipedia.json"
input_file = ""
process_files(input_file, output_file)

In [ ]:
import json
from datasets import Dataset, DatasetDict

with open("/content/scraped_wiki_articles_8_2024.json", "r", encoding="utf-8") as file:
    data = json.load(file)

for entry in data:
    entry["text"] = entry.pop("content")

dataset = Dataset.from_list(data)

dataset_2023 = DatasetDict({"train": dataset})

print(dataset_2023)

In [15]:
import json
import re

full_scores = []
all_scores = []
non_ai = 0
ai = 0
def ai_score(lines, truncate_to=5000):
    print(f"number of lines: {len(lines)}")
    global non_ai, ai
    scores = []
    for ind, line in enumerate(lines):

        print(len(line))
        full_score = llmdet.detect(' '.join(line.strip().split()[:truncate_to]))[0]
        full_scores.append(full_score)
        score = full_score['Human_write']
        print(f" combined: {full_score} \n {score}, {ind}")
        scores.append((score, ind))

        if max(full_score, key=full_score.get) != 'Human_write' and score < 4e-3:
            print("~~~ ai article ~~~")
            ai += 1
        else:
            non_ai += 1

    return scores

def clean_text(text):
    # Replace all newlines with a unique string
    text = text.replace('\n', '!@#').replace('\t', ' ')
    # Split the text by the unique string
    split_text = text.split('!@#')
    # Remove elements with 5 or fewer words
    split_text = [segment for segment in split_text if len(segment.split()) > 5]
    # Rejoin the text
    text = ' '.join(split_text)
    # Remove "References" and "External links" sections
    text = re.sub(r'== References ==.*|== External links ==.*', '', text, flags=re.DOTALL)

    return text.strip()

def extract_title_text_and_url_legacy():
    titles, texts, urls = [], [], []

    for i in range(850):
    #   title = legacy_dataset["train"][i]["title"]
    #   content = legacy_dataset["train"][i]["text"]
    #   url = legacy_dataset["train"][i]["url"]
    #   cleaned_text = clean_text(content)

    #   for new
      title = dataset_2023["train"][i]["title"]
      content = dataset_2023["train"][i]["text"]
      url = dataset_2023["train"][i]["url"]
      cleaned_text = clean_text(content)

      if title and cleaned_text:
          titles.append(title)
          texts.append(cleaned_text)
          urls.append(url)

    return titles, texts, urls

def process_files(file_path, output_file):
    titles, texts, urls = extract_title_text_and_url_legacy()
    texts = [' '.join(text.strip().split()[:400]) for text in texts]
    final_texts, final_titles, final_urls = [],[],[]
    for ind, text in enumerate(texts):
        if len(text.split()) >= 100:
            final_texts.append(texts[ind])
            final_titles.append(titles[ind])
            final_urls.append(urls[ind])

    print(f"number of texts: {len(texts)}")

    bino_scores = ai_score(texts)
    all_scores.append(bino_scores)

    with open(output_file, 'w', encoding='utf-8') as outfile:
        for title, score, url, text in zip(final_titles, bino_scores, final_urls, final_texts):
            outfile.write(f"{title}, {score[0]}\n")


In [ ]:
output_file = "/content/2024-wikipedia.json"
input_file = ""
process_files(input_file, output_file)

# Legacy Articles

In [ ]:
print(all_scores)
print(ai)
print(non_ai)

In [ ]:
import numpy as np

score_nums = []

for score, _ in all_scores[0]:
  # print(score)
  score_nums.append(score)

mean = np.mean(score_nums)
median = np.median(score_nums)
std = np.std(score_nums)

print(f"Mean: {mean}")
print(f"Median: {median}")
print(f"Std dev: {std}")

# New Articles

In [ ]:
# input_file = "/content/scraped_wiki_articles_2_2025.json"
output_file = "/content/wikipedia_new.json"

process_files(input_file, output_file)
print(all_scores)
print(ai)
print(non_ai)

# Scraped Wiki Articles 8_2023

In [ ]:
import json
import re

full_scores = []
all_scores = []

# Your llmdet function needs to be defined or imported here.

filenames = [
    # "CC-MAIN-2022-05.json",
    # "CC-MAIN-2023-40.json",
    # "CC-MAIN-2024-33.json",
    "CC-MAIN-2025-08.json",
]

def clean_text(text):
    text = text.replace('\n', '!@#').replace('\t', ' ')
    split_text = text.split('!@#')
    split_text = [segment for segment in split_text if len(segment.split()) > 5]
    text = ' '.join(split_text)
    text = re.sub(r'== References ==.*|== External links ==.*', '', text, flags=re.DOTALL)
    return text.strip()

def ai_score(lines, truncate_to=5000):
    non_ai, ai = 0, 0
    scores = []
    for ind, line in enumerate(lines):
        full_score = llmdet.detect(' '.join(line.strip().split()[:truncate_to]))[0]
        full_scores.append(full_score)
        score = full_score['Human_write']
        scores.append((score, ind))

        if max(full_score, key=full_score.get) != 'Human_write' and score < 4e-3:
            ai += 1
        else:
            non_ai += 1

    return scores, non_ai, ai

def process_files(file_names):
    for file_name in file_names:

        with open("/content/" + file_name, 'r', encoding='utf-8') as file:
            data = json.load(file)

        # texts = [clean_text(item["text"]) for item in data['train'] if len(item["text"].split()) >= 100]
        texts = [clean_text(item["text"]) for item in data['sites'] if len(item["text"].split()) >= 100]
        print(f"Processing {file_name}, number of texts: {len(texts)}")

        bino_scores, non_ai_count, ai_count = ai_score(texts)
        all_scores.append(bino_scores)

        print(f"{file_name}: Non-AI articles: {non_ai_count}, AI articles: {ai_count}")

process_files(filenames)


# Common Crawl

wiki

Legacy: 249 AI 957 Human

New: 498 AI 1914 Human

8/2023: 405 AI 595 Human

8/2024: 309 AI 540 Human

---

crawl

2024-33: 346 AI 531 Human

2023-40: 340 AI 572 Human

2022-05: 382 AI 544 Human

2025-08: 339 AI 570 Human